In [37]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
import scipy.io as io
from perm_test import calc_perm_test

from constants import feature_names

pd.options.display.max_columns = 500
pd.options.display.max_columns = 500

In [2]:
PATH = 'clusterData_no_light_29_03_22/0'

df = None
files = os.listdir(PATH)
for file in sorted(files):
    if df is None:
        df = pd.read_csv(PATH + '/' + file)
        if df.label[0] < 0:
            df = None
    else:
        temp = pd.read_csv(PATH + '/' + file)
        if temp.label[0] < 0:
            continue
        df = df.append(temp, ignore_index=True )

In [3]:
 df[df.region > 1].label

184    0.0
202    0.0
203    0.0
Name: label, dtype: float64

In [4]:
for c in ['trough2peak', 'd_kl_mid', 'fzc_graph_slowest_path']:
    col_pyr = df[df.label == 1][c].to_numpy()
    col_pv = df[df.label == 0][c].to_numpy()
    
    df_ncx = df[df.region == 0]
    col_pyr_ncx = df_ncx[df_ncx.label == 1][[c]].to_numpy()
    col_pv_ncx = df_ncx[df_ncx.label == 0][[c]].to_numpy()
    
    df_ca1 = df[df.region == 1]
    col_pyr_ca1 = df_ca1[df_ca1.label == 1][[c]].to_numpy()
    col_pv_ca1 = df_ca1[df_ca1.label == 0][[c]].to_numpy()
    
    c_ind = abs(np.median(col_pyr) - np.median(col_pv)) / (np.median(col_pyr) + np.median(col_pv))
    p_val_all = calc_perm_test(col_pyr, col_pv, 5000) #stats.mannwhitneyu(col_pyr, col_pv).pvalue#
    c_ind_ncx = abs(np.median(col_pyr_ncx) - np.median(col_pv_ncx)) / (np.median(col_pyr_ncx) + np.median(col_pv_ncx))
    p_val_ncx = calc_perm_test(col_pyr_ncx, col_pv_ncx, 5000) #stats.mannwhitneyu(col_pyr_ncx, col_pv_ncx).pvalue#
    c_ind_ca1 = abs(np.median(col_pyr_ca1) - np.median(col_pv_ca1)) / (np.median(col_pyr_ca1) + np.median(col_pv_ca1))
    p_val_ca1 = calc_perm_test(col_pyr_ca1, col_pv_ca1, 5000) #stats.mannwhitneyu(col_pyr_ca1, col_pv_ca1).pvalue#
    
    print(f"all {c}-index={c_ind:.3f} ({p_val_all:.3f}); NCX:{c_ind_ncx:.3f} ({p_val_ncx:.3f}); CA1:{c_ind_ca1:.3f} ({p_val_ca1:.3f})")
    
    

all trough2peak-index=0.456 (0.000); NCX:0.391 (0.000); CA1:0.482 (0.000)
all d_kl_mid-index=0.892 (0.000); NCX:0.157 (0.257); CA1:0.907 (0.000)
all fzc_graph_slowest_path-index=0.359 (0.000); NCX:0.081 (0.077); CA1:0.396 (0.000)


In [5]:
PATH = 'ml/results_rf_300322_region.csv'

In [6]:
results = pd.read_csv(PATH, index_col=0)
complete = results[results.restriction == 'complete']
complete = complete.dropna(how='all', axis=1)
grouped_complete = complete.groupby(by=['restriction', 'modality', 'chunk_size'])
grouped_complete[['auc', 'dev_auc']].quantile(q=[0.25, 0.5, 0.75])

auc   dev_auc
restriction modality      chunk_size                         
complete    morphological 0.0        0.25  0.981237  0.990367
                                     0.50  0.988337  1.000000
                                     0.75  0.992901  1.000000
                          25.0       0.25  0.993915  0.998120
                                     0.50  0.994929  1.000000
                                     0.75  0.995943  1.000000
                          800.0      0.25  0.982505  0.995536
                                     0.50  0.989351  0.999530
                                     0.75  0.993915  1.000000
            spatial       0.0        0.25  0.591912  0.756696
                                     0.50  0.613590  0.807801
                                     0.75  0.629817  0.866659
                          25.0       0.25  0.712221  0.878524
                                     0.50  0.745943  0.915883
                                     0.75  0.760396  0.941494
                          800.0      0.25  0.631085  0.831062
                                     0.50  0.645030  0.869831
                                     0.75  0.661765  0.895677
            temporal      0.0        0.25  0.752789  0.981673
                                     0.50  0.766988  0.992951
                                     0.75  0.778144  0.997180
                          25.0       0.25  0.633874  0.984962
                                     0.50  0.637931  0.989662
                                     0.75  0.640974  0.995066
                          800.0      0.25  0.742647  0.988957
                                     0.50  0.755071  0.993421
                                     0.75  0.768509  0.997180

In [7]:
df0 = complete[complete.chunk_size == 0]
wf_ca1 = complete[complete.modality == 'morphological'].dev_auc.to_numpy()
wf_ncx = complete[complete.modality == 'morphological'].auc.to_numpy()

temp_ca1 = complete[complete.modality == 'temporal'].dev_auc.to_numpy()
temp_ncx = complete[complete.modality == 'temporal'].auc.to_numpy()

spat_ca1 = complete[complete.modality == 'spatial'].dev_auc.to_numpy()
spat_ncx = complete[complete.modality == 'spatial'].auc.to_numpy()

statistic, p_val = stats.mannwhitneyu(wf_ca1, wf_ncx)
print(f"Mann-Whitney statistical test results for WF are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(temp_ca1, temp_ncx)
print(f"Mann-Whitney statistical test results for temporal are p-value={p_val} (statistic={statistic})")
print()

statistic, p_val = stats.mannwhitneyu(spat_ca1, spat_ncx)
print(f"Mann-Whitney statistical test results for spatial are p-value={p_val} (statistic={statistic})")
print()

Mann-Whitney statistical test results for WF are p-value=1.784759863608574e-18 (statistic=4780.0)

Mann-Whitney statistical test results for temporal are p-value=5.276520204865512e-51 (statistic=0.0)

Mann-Whitney statistical test results for spatial are p-value=3.5895681264677576e-46 (statistic=569.0)



In [16]:
# data for kruskal wallis on matlab
cs_0 = complete[complete.chunk_size == 0]
aucs = cs_0.auc.to_numpy()
dev_aucs = cs_0.dev_auc.to_numpy()
N = len(aucs)
all_aucs = np.concatenate((aucs, dev_aucs))
g1 = np.concatenate((np.ones(N), 2 * np.ones(N)))
g2 = np.repeat([[1, 2, 3]], (2 * N) // 3, axis=0).flatten()

In [17]:
m_dict = {'data': all_aucs, 'g1': g1, 'g2': g2}
io.savemat('region_mat.mat', m_dict)

In [18]:
diff = complete.dev_auc - complete.auc
complete['auc_diff'] = diff

In [24]:
diff_0 = complete[complete.chunk_size == 0]
wf_diff = complete[complete.modality == 'morphological'].auc_diff.to_numpy()
temp_diff = complete[complete.modality == 'temporal'].auc_diff.to_numpy()
spat_diff = complete[complete.modality == 'spatial'].auc_diff.to_numpy()

wf_median, wf_prec25, wf_prec75 = np.percentile(wf_diff, [50, 25, 75])
print(f"Median of WF column is {wf_median: .3f} [{wf_prec25: .3f}, {wf_prec75: .3f}]")
temp_median, temp_prec25, temp_prec75 = np.percentile(temp_diff, [50, 25, 75])
print(f"Median of temporal column is {temp_median: .3f} [{temp_prec25: .3f}, {temp_prec75: .3f}]")
spat_median, spat_prec25, spat_prec75 = np.percentile(spat_diff, [50, 25, 75])
print(f"Median of spatial column is {spat_median: .3f} [{spat_prec25: .3f}, {spat_prec75: .3f}]\n")

statistic, p_val = stats.mannwhitneyu(wf_diff, temp_diff)
print(f"Mann-Whitney statistical test results for wf and temp are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.mannwhitneyu(wf_diff, spat_diff)
print(f"Mann-Whitney statistical test results for wf and spatial are p-value={p_val} (statistic={statistic})")
statistic, p_val = stats.mannwhitneyu(spat_diff, temp_diff)
print(f"Mann-Whitney statistical test results for temporal and spatial p-value={p_val} (statistic={statistic})")

Median of WF column is  0.006 [ 0.003,  0.010]
Median of temporal column is  0.246 [ 0.222,  0.342]
Median of spatial column is  0.197 [ 0.155,  0.236]

Mann-Whitney statistical test results for wf and temp are p-value=5.296901957290956e-51 (statistic=0.0)
Mann-Whitney statistical test results for wf and spatial are p-value=5.296769359122747e-51 (statistic=0.0)
Mann-Whitney statistical test results for temporal and spatial p-value=1.1090521091813217e-16 (statistic=5082.0)


In [57]:
def calc_stat(vals, label, region):
    vals_ca1 = vals[region == 1]
    vals_ncx = vals[region == 0]
    ca1_diff = np.median(vals_ca1[label[region == 1] == 1]) - np.median(vals_ca1[label[region == 1] == 0])
    ncx_diff = np.median(vals_ncx[label[region == 0] == 1]) - np.median(vals_ncx[label[region == 0] == 0])
    statistic = ca1_diff - ncx_diff
    return statistic

def special_perm_test(vals, label, region, reps=1000):
    np.random.seed(0)
    stat_val = calc_stat(vals, label, region)
    region = region.copy()

    rand_stats = []
    for i in range(reps):
        np.random.shuffle(region)
        rand_stats.append(calc_stat(vals, label, region))
    rand_stats = np.asarray(rand_stats)
    
    print(stat_val, np.percentile(rand_stats, [25, 50, 75]))
        
    if stat_val < 0:
        p_val = (np.sum(stat_val >= rand_stats) + 1) / (reps + 1)
        return p_val
    elif stat_val > 0:
        p_val = (np.sum(stat_val <= rand_stats) + 1) / (reps + 1)
        return p_val
    else:
        print('stat_val == 0')
        return 1

In [58]:
df = df[df.region <= 1]
for fet in feature_names:
    p_val = special_perm_test(df[fet].to_numpy(), df.label.to_numpy(), df.region.to_numpy())
    print(f"for feature {fet} test p-val is {p_val}")

0.0 [0. 0. 0.]
stat_val == 0
for feature spatial_dispersion_count test p-val is 1
0.014242568343064865 [-0.01042649 -0.00061951  0.00929932]
for feature spatial_dispersion_sd test p-val is 0.15584415584415584
-0.04779239782484135 [-0.19401249 -0.00112618  0.19968871]
for feature spatial_dispersion_area test p-val is 0.43456543456543456
0.15000000000000036 [-2.5        -0.29166667  1.7547619 ]
for feature dep_red test p-val is 0.4385614385614386
0.258967249180033 [-0.34304922 -0.03605437  0.28289017]
for feature dep_sd test p-val is 0.26573426573426573
36.42261904761905 [-36.90178571   3.6577381   60.38869048]
for feature fzc_red test p-val is 0.33466533466533466
-0.39626777107252353 [-1.90718552 -0.05829491  2.46977581]
for feature fzc_sd test p-val is 0.44655344655344653
-11.100000000000001 [-11.72321429   0.3375      15.53363095]
for feature szc_red test p-val is 0.2597402597402597
-0.20750129077506818 [-0.77132561 -0.02556538  0.94619475]
for feature szc_sd test p-val is 0.449550449